# Non-negative PARAFAC analysis on bike sharing data

## Import all dependencies

In [1]:
# General data analysis and numerics
import numpy as np
import pandas as pd
import xarray

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact
from wordcloud import WordCloud

# Tensor algorithms
import tensorly as tl
import tensorly.decomposition
from component_vis.postprocessing import postprocess

# General utilities
import calendar
from tqdm import trange

## Download bike share data using the Oslo Bysykkel API

We count the number of trips that start at each station for each day of week, our of day and month.

**Info about dataset:** https://oslobysykkel.no/apne-data

In [2]:
year = 2020

data = []
start_data = []
end_data = []
for month in trange(12):
    month += 1
    df = pd.read_csv(f"https://data.urbansharing.com/oslobysykkel.no/trips/v1/{year}/{month:02d}.csv")
    station_names = sorted(set(df["start_station_name"]) | set(df["end_station_name"]))

    df["ended_at"] = df["ended_at"].map(lambda x: x if "." in x else ".000000+".join(x.split("+")))
    df["ended_at"] = pd.to_datetime(df["ended_at"], format="%Y-%m-%d %H:%M:%S.%f+00:00")
  
    end_time = df["ended_at"].dt

    df["trip"] = 1
    df["Day"] = end_time.day
    df["Day of week"] = end_time.dayofweek
    df["Hour"] = end_time.hour
    df["Month"] = end_time.month
    df["Year"] = end_time.year
    df = df.rename({"end_station_name": "End station name"}, axis=1)

    end = df.groupby(["End station name", "Year", "Month", "Day of week", "Hour"]).sum()[["trip"]]
    end_data.append(end)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:21<00:00,  6.76s/it]


## Generate data tensor where each element corresponds to the number of trips that ended at a given station at the given time

In [3]:
grouped = pd.concat(end_data).groupby(level=(0, 1, 2, 3, 4)).sum()
grouped = grouped.loc[(slice(None), 2020),]

dataset = xarray.Dataset.from_dataframe(grouped).to_array().squeeze()
dataset.values[np.isnan(dataset.values)] = 0
dataset

<xarray.DataArray (End station name: 255, Month: 10, Day of week: 7, Hour: 24)>
array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  2.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.],
         ...,
...
         ...,
         [ 0.,  0.,  0., ...,  0.,  5.,  1.],
         [ 0.,  0.,  0., ...,  0.,  2.,  2.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  1.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  3.,  0.,  1.],
         [ 1.,  0.,  0., ...,  0.,  1.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]]])
Coordinates:
  * End station name  (End station name) object '7 Juni Plassen' ... 'Økernve...
    Year              int64 2020
  * Month             (Month) int64 3 4 5 6 7 8 9 10 11 12
  * Day of week       (Day of week) int64 0 1 2 3 4 5 6
  * Hour              (Hour) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
    variable          <U4 'trip'

## We create additional utilities to map station names to coordinates

In [4]:
station_to_lat = {}
station_to_lon = {}
for _, trip in df.iterrows():
    key = trip['start_station_name']
    lat, lon = trip['start_station_latitude'], trip['start_station_longitude']
    station_to_lat[key] = lat
    station_to_lon[key] = lon

## We fit a three-component PARAFAC model to the data

In [5]:
rank = 3
parafac_model = tl.decomposition.non_negative_parafac_hals(
    dataset.values,
    rank,
    1000,
    init='random',
    tol=1e-8
)
parafac_model = postprocess(parafac_model, dataset=dataset)

## Visualise the extracted components

In [6]:
def visualise_component(component):
    A, B, C, D = parafac_model[1]
    fig, axes = plt.subplots(1, 4, figsize=(16, 2))
  
    # Generate word cloud data
    wc = WordCloud(background_color="black", max_words=1000, colormap="Pastel1")
    frequencies = A[component].to_dict()
    wc.generate_from_frequencies(frequencies)

    # Plot components
    axes[0].imshow(wc, interpolation="bilinear")
    B[component].plot(ax=axes[1])
    C[component].plot(ax=axes[2])
    D[component].plot(ax=axes[3])
    plt.show()

    # Create component dataframe with coordinates
    A_coord = A.copy()
    A_coord['lat'] = pd.DataFrame([{'lat': lat, 'end_station_name': key} for key, lat in station_to_lat.items()]).set_index("end_station_name")
    A_coord['lon'] = pd.DataFrame([{'lon': lon, 'end_station_name': key} for key, lon in station_to_lon.items()]).set_index("end_station_name")

    # Add density map plot
    fig = px.density_mapbox(A_coord.reset_index(), lat="lat", lon="lon", z=component, zoom=10, opacity=0.5)
    fig.update_layout(mapbox_style="carto-positron",)
    fig.show()

In [7]:
interact(visualise_component, component=range(rank))

interactive(children=(Dropdown(description='component', options=(0, 1, 2), value=0), Output()), _dom_classes=(…